In [225]:
import numpy as np
import scipy as scp
import pandas as ps
#from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from scipy.stats import pearsonr, spearmanr

def rmse(a, b):
    return np.sqrt(np.sum((np.array(a)-np.array(b))**2)/len(a))

def r2(a, b):
    return 1 - np.sum((np.array(a)-np.array(b))**2)/np.sum((np.array(a)-np.mean(a))**2)

In [226]:
co_structure = [1.00000, 0.99900, 0.99800, 0.99700, 0.99600, 0.99500, 0.99000, 0.98500, 0.98000, 0.97500, 0.97000,
                0.96500, 0.96000, 0.95500, 0.95000, 0.90000, 0.85000, 0.80000, 0.75000, 0.70000, 0.65000, 0.60000,
                0.55000, 0.50000, 0.45000, 0.40000, 0.35000, 0.30000, 0.25000, 0.2, 0.15, 0.1, 0.05, 0.0]

# file paths should be changed to actual placement
SS = ps.read_excel('Structure_Similarity.xls')
train = ps.read_table('../../rawdata/training.tsv', index_col=0)
test = ps.read_table('../../rawdata/test.tsv', index_col=0)

Grid search for the parameters. Different number of trees, tree depths and learning rates are tested. For each parameter, the calculations are run 10 times with different seeds and the mean performance (Rp, Rs and RMSE) is reported. Tests are run with the whole training set, performances are evaluated on the test set.

In [211]:
for md in [5,6,7,8,9,10]:                                   # maximum depth
    for lr in [0.1,0.05,0.01]:                              # learning rate
        for nt in [200,300,400,500,600,700,800,900,1000]:   # number of trees
            ppreds = []; rps = 0; rss = 0; rmses = 0
            for seed in np.random.randint(1, 5000, size=10):
                RGR = xgb.XGBRegressor(reg_lambda=0.001, max_depth=md, learning_rate=lr, subsample=0.5, n_estimators=nt,colsample_bytree=0.56, silent=1, random_state=seed)
                RGR.fit(train.loc[:,'CC':], train.loc[:, 'pbindaff'], eval_metric='mae')
                preds = RGR.predict(test.loc[:,'CC':])
                ppreds.append(preds)
                rps += pearsonr(test['pbindaff'], preds)[0]
                rss += spearmanr(test['pbindaff'], preds)[0]
                rmses += rmse(test['pbindaff'], preds)
            print(md, lr, nt, rps/10, rss/10, rmses/10, sep='\t')

5 	 0.1 	 200 	 0.7763689273850372 	 0.7717935647260697 	 1.5040265442350667
5 	 0.1 	 300 	 0.7755995682118993 	 0.7713458229774122 	 1.5049448928051263
5 	 0.1 	 400 	 0.7686129276051524 	 0.7635226020677394 	 1.5233887609038261
5 	 0.1 	 500 	 0.7665571163040918 	 0.7605502539437666 	 1.5302884774129215
5 	 0.1 	 600 	 0.7649031409877578 	 0.7594159694523636 	 1.5334933766248604
5 	 0.1 	 700 	 0.7678806188718599 	 0.7645290297070045 	 1.5236237010593983
5 	 0.1 	 800 	 0.7665301062503582 	 0.7616227949046751 	 1.5276976437618457
5 	 0.1 	 900 	 0.7720345965091077 	 0.769519978575648 	 1.511525763922005
5 	 0.1 	 1000 	 0.7688298076329747 	 0.7630745366308485 	 1.5210519852647981
5 	 0.05 	 200 	 0.7857881664503452 	 0.7794575310249783 	 1.4980212583135615
5 	 0.05 	 300 	 0.7796434934494272 	 0.7773251538663165 	 1.5023544223686778
5 	 0.05 	 400 	 0.7848672956465554 	 0.7812471223446172 	 1.4853547806173588
5 	 0.05 	 500 	 0.7815419511608018 	 0.7763998101294829 	 1.4923108136279

8 	 0.01 	 900 	 0.8036668695849988 	 0.7947611870053011 	 1.440696472003507
8 	 0.01 	 1000 	 0.8052892846108154 	 0.7993057698002781 	 1.4344484577350622
9 	 0.1 	 200 	 0.7778730758189456 	 0.7762126374086002 	 1.496489183273469
9 	 0.1 	 300 	 0.7910625969588805 	 0.783651478541276 	 1.4616336522176097
9 	 0.1 	 400 	 0.7763337914580598 	 0.7672336876620708 	 1.5018079566826077
9 	 0.1 	 500 	 0.7826079806853565 	 0.7739171213803752 	 1.486919304772582
9 	 0.1 	 600 	 0.7797699854398112 	 0.7705719653337446 	 1.491564097329889
9 	 0.1 	 700 	 0.7837396925211559 	 0.7761712053147428 	 1.4834781734795146
9 	 0.1 	 800 	 0.7804978892915044 	 0.7761741185088421 	 1.4909701167861926
9 	 0.1 	 900 	 0.7806529717733868 	 0.7731121896663146 	 1.4906692888091655
9 	 0.1 	 1000 	 0.7858387978197967 	 0.7780094307914291 	 1.4733586144222373
9 	 0.05 	 200 	 0.7944906545050048 	 0.7883298254986482 	 1.457138784298727
9 	 0.05 	 300 	 0.7990363670914158 	 0.7915787652179421 	 1.4424659987766109

Results for Table 1: Performance of XGBoost trained on different structural similarity levels

In [219]:
cutoffs = {}
for co in co_structure:
    cutoffs[co] = SS[SS.max(axis=1)<=co].index
    if len(cutoffs[co])>0:                            # to exclude excepions with empty training sets
        rp = 0; rs = 0; rmses = 0
        for seed in np.random.randint(0, 9999, size=10):
            RGR = xgb.XGBRegressor(reg_lambda=0.001, max_depth=9, learning_rate=0.01, subsample=0.5, n_estimators=1000,colsample_bytree=0.58, silent=1, random_state=seed)
            RGR.fit(train.loc[cutoffs[co],'CC':], train.loc[cutoffs[co], 'pbindaff'], eval_metric='rmse')
            preds = RGR.predict(test.loc[:,'CC':])
            if len(cutoffs[co])==1:                  # to exclude exceptions with Rp and Rs calculations on 1 example
                rp = np.nan
                rs = np.nan
            else:
                rp += pearsonr(test['pbindaff'], preds)[0]
                rs += spearmanr(test['pbindaff'], preds)[0]
            rmses += rmse(test['pbindaff'], preds)
        print(co, len(cutoffs[co]), rp/10, rs/10, rmses/10, sep='\t')

1.0	1105	0.8064938692264734	0.799103626498607	1.4290335124018092
0.999	964	0.7681999182337883	0.7762874810811381	1.555279280180205
0.998	867	0.742447577504785	0.7557403277195742	1.6168420513847503
0.997	810	0.7276670475637596	0.7458781946285935	1.6656254319916255
0.996	793	0.7301134740176046	0.7472549216067087	1.6623155007957098
0.995	779	0.7195822990314565	0.7343843491537468	1.6835759460493729
0.99	710	0.7042166654415805	0.7161143332817667	1.7120752990654626
0.985	685	0.6884181578886613	0.6993049605623745	1.7452188576300283
0.98	645	0.6832728753996881	0.6965559573193515	1.7516980700453035
0.975	605	0.6670286901399132	0.6901103535304355	1.7863613357775758
0.97	582	0.6654912538419893	0.6843574425606967	1.7944710556141552
0.965	567	0.668695368953687	0.6861759230551542	1.7872570149133602
0.96	563	0.6689946140997365	0.687385869671082	1.7886647264740199
0.955	548	0.6665662837406676	0.6837101470162341	1.7910572411232295
0.95	540	0.6596826512953975	0.6765360828583653	1.805498115660423
0.9	510

Results for Table 3: Performance of XGBoost trained on different structural similarity levels, requiring certain similarity level in training examples

In [221]:
cutoffs = {}
for co in reversed(co_structure):
    cutoffs[co] = SS[SS.max(axis=1)>co].index
    if len(cutoffs[co])>0:
        rp = 0; rs = 0; rmses = 0
        for seed in np.random.randint(0, 9999, size=10):
            RGR = xgb.XGBRegressor(reg_lambda=0.001, max_depth=9, learning_rate=0.01, subsample=0.5, n_estimators=1000,colsample_bytree=0.58, silent=1, random_state=seed)
            RGR.fit(train.loc[cutoffs[co],'CC':], train.loc[cutoffs[co], 'pbindaff'], eval_metric='rmse')
            preds = RGR.predict(test.loc[:,'CC':])
            if len(cutoffs[co])==1:
                rp = np.nan
                rs = np.nan
            else:
                rp += pearsonr(test['pbindaff'], preds)[0]
                rs += spearmanr(test['pbindaff'], preds)[0]
            rmses += rmse(test['pbindaff'], preds)
        print(co, len(cutoffs[co]), rp/10, rs/10, rmses/10, sep='\t')

0.0	1105	0.8089740249363953	0.8007750716131058	1.4227737996088554
0.05	1105	0.8073400480761783	0.7996738842435537	1.42726749391563
0.1	1105	0.8070595772566245	0.7986052275414444	1.4265910361770977
0.15	1105	0.8070351192906617	0.8003902063037591	1.4275872490498025
0.2	1105	0.8085592954250742	0.8016470067914513	1.4226888238571378
0.25	1104	0.80658766792171	0.800232731978278	1.4291598097644096
0.3	1096	0.8069550984858112	0.7977867009215867	1.4283006113392676
0.35	1062	0.8048633508803655	0.7999430310095098	1.432060813808477
0.4	989	0.8055551482516478	0.802468122917175	1.4286998553527348
0.45	883	0.7973055322660748	0.7900712681935897	1.4520167218398703
0.5	811	0.7919868378354602	0.7874493935041798	1.4675212331178638
0.55	775	0.7883267423672481	0.7828528587477641	1.4776392632908089
0.6	734	0.7938797256523636	0.7869318969412545	1.4627177349796943
0.65	705	0.7971988492033675	0.7919265682245804	1.4559761713196449
0.7	689	0.8004396706818078	0.7929662548298131	1.446672582186558
0.75	662	0.7953634

In [222]:
co_sequence = [1.00000, 0.99900, 0.99800, 0.99700, 0.99600, 0.99500, 0.99000, 0.98500, 0.98000, 0.97500, 0.97000,
                0.96500, 0.96000, 0.95500, 0.95000, 0.90000, 0.85000, 0.80000, 0.75000, 0.70000, 0.65000, 0.60000,
                0.55000, 0.50000, 0.45000, 0.40000, 0.35000, 0.30000, 0.25000, 0.2, 0.15, 0.1, 0.05, 0.0]

# file paths should be changed to actual placement
SS = ps.read_excel('Sequence_Similarity.xls')
train = ps.read_table('../../rawdata/training.tsv', index_col=0)
test = ps.read_table('../../rawdata/test.tsv', index_col=0)

Results for Table 2: Performance of XGBoost trained on different sequence similarity levels

In [223]:
cutoffs = {}
for co in co_sequence:
    cutoffs[co] = SS[SS.max(axis=1)<=co].index
    if len(cutoffs[co])>0:
        rp = 0; rs = 0; rmses = 0
        for seed in np.random.randint(0, 9999, size=10):
            RGR = xgb.XGBRegressor(reg_lambda=0.001, max_depth=9, learning_rate=0.01, subsample=0.5, n_estimators=1000,colsample_bytree=0.58, silent=1, random_state=seed)
            RGR.fit(train.loc[cutoffs[co],'CC':], train.loc[cutoffs[co], 'pbindaff'], eval_metric='rmse')
            preds = RGR.predict(test.loc[:,'CC':])
            if len(cutoffs[co])==1:
                rp = np.nan
                rs = np.nan
            else:
                rp += pearsonr(test['pbindaff'], preds)[0]
                rs += spearmanr(test['pbindaff'], preds)[0]
            rmses += rmse(test['pbindaff'], preds)
        print(co, len(cutoffs[co]), rp/10, rs/10, rmses/10, sep='\t')

1.0	1105	0.8072835931702025	0.8000551080602764	1.4268917604872946
0.999	786	0.7226311399064146	0.7434743239642845	1.67904413413993
0.998	785	0.7197256137875907	0.7410986950983276	1.684377546036909
0.997	781	0.7191205683626324	0.7390265239510521	1.6843233818635515
0.996	746	0.7105173335795325	0.7319819350864873	1.7044499414373255
0.995	726	0.7007204083195392	0.7203364416743583	1.728174577400417
0.99	692	0.6890337465228481	0.7067707487404695	1.7426718900147122
0.985	678	0.6832122597424439	0.7008695076378391	1.7580321153995295
0.98	670	0.6761684549949436	0.6959876417038046	1.7723993209188738
0.975	640	0.6635637367925572	0.6857047948316435	1.7927937867924828
0.97	638	0.6617818969738469	0.6837662260026465	1.7937654138624635
0.965	606	0.6738569374419148	0.6958763738736211	1.772332282038494
0.96	606	0.6751963210590801	0.6985998057462166	1.7691584377220182
0.955	584	0.6612576192239019	0.6852623939388348	1.8015410285822735
0.95	584	0.6642815387095273	0.6873261491920453	1.7946529275339536
0.9	56

Results for Table 4: Performance of XGBoost trained on different sequence similarity levels, requiring certain similarity level in training examples

In [224]:
cutoffs = {}
for co in reversed(co_sequence):
    cutoffs[co] = SS[SS.max(axis=1)>co].index
    if len(cutoffs[co])>0:
        rp = 0; rs = 0; rmses = 0
        for seed in np.random.randint(0, 9999, size=10):
            RGR = xgb.XGBRegressor(reg_lambda=0.001, max_depth=9, learning_rate=0.01, subsample=0.5, n_estimators=1000,colsample_bytree=0.58, silent=1, random_state=seed)
            RGR.fit(train.loc[cutoffs[co],'CC':], train.loc[cutoffs[co], 'pbindaff'], eval_metric='rmse')
            preds = RGR.predict(test.loc[:,'CC':])
            if len(cutoffs[co])==1:
                rp = np.nan
                rs = np.nan
            else:
                rp += pearsonr(test['pbindaff'], preds)[0]
                rs += spearmanr(test['pbindaff'], preds)[0]
            rmses += rmse(test['pbindaff'], preds)
        print(co, len(cutoffs[co]), rp/10, rs/10, rmses/10, sep='\t')

0.0	1105	0.8057633012707986	0.7968840962831397	1.431953828014297
0.05	1105	0.8066569860002468	0.7992921748944812	1.428947073329904
0.1	1105	0.8078242260446562	0.7991280649602184	1.4251922253690044
0.15	1104	0.8083069794311598	0.801598129868229	1.4250906291133845
0.2	1091	0.8075754921141527	0.8010325656026647	1.4254447538827
0.25	1049	0.8022187031079966	0.7961806408302065	1.440886331699782
0.3	924	0.7972391663246878	0.7909774334026024	1.4534634722607598
0.35	755	0.7942255101399464	0.7838236806813708	1.4581407669118487
0.4	654	0.7971676254903616	0.7891413118994323	1.4527479180168301
0.45	597	0.7972236137465698	0.789285272241175	1.4561166001591925
0.5	583	0.7984201106079734	0.7897160203575864	1.4497844092265688
0.55	570	0.8017255824038131	0.7947299510907914	1.4414070148113023
0.6	563	0.8028966785572471	0.7942208704219308	1.4407821762808222
0.65	560	0.8014417323594569	0.7921204574763036	1.4461813503483318
0.7	553	0.8004784085876709	0.7922845674105665	1.4462948528678918
0.75	552	0.800134480